In [1]:
from timm import create_model
from timm.models import ResNet
import torch

C:\Users\enguye17\PycharmProjects\QuaRS\new_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import main_helper

In [3]:
import importlib
importlib.reload(main_helper)

<module 'main_helper' from 'C:\\Users\\enguye17\\PycharmProjects\\QuaRS\\main_helper.py'>

In [32]:
import argparse

config = argparse.Namespace(
    dataset='CIFAR10',
    model='VIT_Tiny',
    batch_size=128,
    learning_rate=0.001,
    epoch=50,
    technique='LoRITa',
    rectangular_mode=True,
    target_rank=4,
    lmbda=0.01,
    depth_lorita=4,
    weight_decay_alpha=0.00,
    dropout=0.0,
    threads=4,
    augmentation=False,
    target_modules=['Q', 'K', 'V','FC1','FC2'],
    DEVICE = torch.device("cuda") 
)

In [33]:
model = main_helper.create_model('timm/vit_tiny_patch16_224', pretrained=False, num_classes=10)
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()


In [34]:
model = main_helper.model_parser((-1,3,224,224), 10, config)
regularizer =  main_helper.configure_model_experiment(model,config)
main_helper.get_model_memory(model)

Total parameters: 5,526,346
Total parameters (M): 5.53M
Total parameters: 5526346
Total memory (MB): 0.00
injecting LoR
Total parameters: 32,068,426
Total parameters (M): 32.07M


(32068426, 32.068426)

In [35]:
cloned_config = argparse.Namespace(
    dataset='CIFAR10',
    model='VIT_Tiny',
    batch_size=128,
    learning_rate=0.001,
    epoch=50,
    technique='Truncate',
    rectangular_mode=True,
    target_rank=4,
    lmbda=0.01,
    depth_lorita=2,
    weight_decay_alpha=0.00,
    dropout=0.0,
    threads=4,
    augmentation=False,
    target_modules=['Q', 'K', 'V','FC1','FC2'],
    DEVICE = torch.device("cuda") 
)

In [36]:
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): LoRITaQKV(
          (q_lorita): LoRITa Module (in_features=192, out_features=192, N=4)
          (k_lorita): LoRITa Module (in_features=192, out_features=192, N=4)
          (v_lorita): LoRITa Module (in_features=192, out_features=192, N=4)
        )
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
 

In [37]:
random_vector = torch.randn(192)
random_vector = random_vector.to(next(model.parameters()).device)

In [38]:
lorita_result = model.blocks[0].attn.qkv(random_vector)
fc1_lorita_result = model.blocks[0].mlp.fc1(random_vector)

In [39]:
regularizer =  main_helper.configure_model_experiment(model,cloned_config)
main_helper.get_model_memory(model)

cleansing LorITA scum
['blocks.0.attn.qkv', 'blocks.0.attn.qkv.q_lorita', 'blocks.0.attn.qkv.k_lorita', 'blocks.0.attn.qkv.v_lorita', 'blocks.0.mlp.fc1', 'blocks.0.mlp.fc2', 'blocks.1.attn.qkv', 'blocks.1.attn.qkv.q_lorita', 'blocks.1.attn.qkv.k_lorita', 'blocks.1.attn.qkv.v_lorita', 'blocks.1.mlp.fc1', 'blocks.1.mlp.fc2', 'blocks.2.attn.qkv', 'blocks.2.attn.qkv.q_lorita', 'blocks.2.attn.qkv.k_lorita', 'blocks.2.attn.qkv.v_lorita', 'blocks.2.mlp.fc1', 'blocks.2.mlp.fc2', 'blocks.3.attn.qkv', 'blocks.3.attn.qkv.q_lorita', 'blocks.3.attn.qkv.k_lorita', 'blocks.3.attn.qkv.v_lorita', 'blocks.3.mlp.fc1', 'blocks.3.mlp.fc2', 'blocks.4.attn.qkv', 'blocks.4.attn.qkv.q_lorita', 'blocks.4.attn.qkv.k_lorita', 'blocks.4.attn.qkv.v_lorita', 'blocks.4.mlp.fc1', 'blocks.4.mlp.fc2', 'blocks.5.attn.qkv', 'blocks.5.attn.qkv.q_lorita', 'blocks.5.attn.qkv.k_lorita', 'blocks.5.attn.qkv.v_lorita', 'blocks.5.mlp.fc1', 'blocks.5.mlp.fc2', 'blocks.6.attn.qkv', 'blocks.6.attn.qkv.q_lorita', 'blocks.6.attn.qkv.k

(5526346, 5.526346)

In [40]:
simplify_result = model.blocks[0].attn.qkv(random_vector)
fc1_simplify_result = model.blocks[0].mlp.fc1(random_vector)

In [41]:
lorita_result-simplify_result

tensor([-4.4703e-08, -3.7253e-08, -2.2352e-08, -5.2154e-08, -2.2352e-08,
        -5.2154e-08,  5.9605e-08, -1.4901e-08, -1.4901e-08,  3.7253e-08,
        -1.4901e-08,  5.9605e-08, -5.2154e-08, -2.2352e-08, -1.6764e-08,
         3.7253e-09,  5.9605e-08, -1.8626e-09,  1.8626e-08, -2.2352e-08,
        -7.4506e-09,  7.4506e-08, -5.2154e-08,  0.0000e+00, -5.9605e-08,
         1.3039e-08, -5.2154e-08,  2.2352e-08,  2.9802e-08, -1.4901e-08,
         4.4703e-08,  0.0000e+00, -1.4901e-08, -1.4901e-08,  6.7055e-08,
         2.9802e-08,  1.1176e-08, -3.1665e-08, -2.9802e-08,  2.2352e-08,
        -1.4901e-08, -1.4901e-08, -4.4703e-08,  0.0000e+00,  2.9802e-08,
         7.4506e-09,  2.2352e-08,  4.4703e-08, -4.8429e-08,  0.0000e+00,
         1.4901e-08,  7.4506e-09, -2.9802e-08, -8.9407e-08,  1.1176e-08,
         3.7253e-09, -2.9802e-08, -4.4703e-08,  0.0000e+00, -3.7253e-09,
         5.5879e-08, -1.4901e-08, -4.0978e-08, -2.2352e-08, -5.9605e-08,
        -7.4506e-09,  0.0000e+00,  0.0000e+00,  0.0

In [42]:
fc1_lorita_result-fc1_simplify_result

tensor([-1.4901e-08, -1.8626e-08, -2.2352e-08,  5.9605e-08,  0.0000e+00,
        -2.9802e-08,  0.0000e+00, -5.2154e-08, -4.8429e-08, -5.2154e-08,
         7.4506e-09,  7.4506e-09,  0.0000e+00,  0.0000e+00,  2.9802e-08,
        -2.9802e-08, -3.7253e-09, -1.4901e-08, -2.9802e-08,  0.0000e+00,
         1.4901e-08, -5.5879e-09, -1.8626e-08, -2.9802e-08, -2.9802e-08,
         4.4703e-08, -6.7055e-08, -4.4703e-08, -7.4506e-08, -1.8626e-08,
        -5.2154e-08, -4.4703e-08,  1.4901e-08,  0.0000e+00,  0.0000e+00,
        -4.0978e-08, -2.9802e-08, -1.4901e-08,  0.0000e+00,  0.0000e+00,
        -8.9407e-08,  2.6077e-08, -2.9802e-08,  2.9802e-08,  2.9802e-08,
         0.0000e+00, -4.4703e-08, -2.2352e-08, -2.2352e-08,  0.0000e+00,
        -7.4506e-09, -2.2352e-08, -2.9802e-08,  1.4901e-08,  2.2352e-08,
         2.9802e-08,  7.4506e-09,  7.4506e-08, -3.7253e-09, -1.4901e-08,
        -1.4901e-08,  4.0978e-08, -1.8626e-08,  5.9605e-08,  0.0000e+00,
         5.9605e-08, -7.4506e-09,  7.4506e-09,  0.0

torch.Size([768])

In [ ]:
model()